In [1]:
# Activation function
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [2]:
# Function to initialize parameters
def initialize_parameters(inputUnits, layerUnits):
    w = np.random.randn(layerUnits, inputUnits)*0.01
    b = np.zeros((layerUnits, 1))
    
    return w, b

In [3]:
# Function to add layers to the neural network
def add_layer(model, inputUnits, layerUnits):
    model.append({"w":initialize_parameters(inputUnits, layerUnits)[0],        
                "b":initialize_parameters(inputUnits, layerUnits)[1]})

In [5]:
# Function to propagate forward and backword through the neural network
def propagate(model, X, Y):
    Z = []
    A = []
    # forward propagation
    for i in range(len(model)):
        W = model[i]["w"]
        b = model[i]["b"]

        if i == 0: 
            Z.append(sigmoid(np.dot(W, X) + b))
            A.append(np.dot(W, X) + b)
        else:
            Z.append(sigmoid(np.dot(W, Z[i-1]) + b))
            A.append(np.dot(W, Z[i-1]) + b)

    m = X.shape[1]
    cost = -1/m*(np.sum(Y*np.log(Z[len(model)-1])+(1-Y)*np.log(1-Z[len(model)-1])))

    # backward propagation
    dA = []
    dZ = []
    dW = []
    db = []

    for i in range(len(model)-1, -1, -1):

        if i == len(model) - 1:
            dA.insert(0, Z[i] - Y)
        else:
            W = model[i+1]["w"]
            dA.insert(0, np.dot(W.T, dA[0])*(sigmoid(A[i])*(1-sigmoid(A[i]))))
            
        if i != 0:
            dW.insert(0, 1/m*np.dot(dA[0], Z[i-1].T))
        else:
            dW.insert(0, 1/m*np.dot(dA[0], X.T))
        
        db.insert(0, 1/m*np.sum(dA[0], axis=1, keepdims=True))
    
        grads = {"dW": dW, "db": db}
    return grads, cost 

In [6]:
# Function to find optimal value of parameters (weights and bias)
def optimize(model, X, Y, iterations, learning_rate, print_cost=False):
    costs = []

    for i in range(iterations):
        grads, cost = propagate(model, X, Y)

        dW = grads["dW"]
        db = grads["db"]

        for j in range(len(model)):
            model[j]["w"] -= learning_rate*dW[j]
            model[j]["b"] -= learning_rate*db[j] 

        if i%100 == 0:
            costs.append(cost)
    
        if print_cost and i%100 == 0:
            print("Cost at the end of %i th iteration is: %.4f" %(i, cost))

    return model, grads, costs

In [7]:
# Predicting output for the give value of model parameters
def predict(model, X):

    for i in range(len(model)):
        W = model[i]["w"]
        b = model[i]["b"]

        if i == 0:
            Z = sigmoid(np.dot(W, X) + b)
        else:
            Z = sigmoid(np.dot(W, Z) + b)
    
    Y_prediction = (Z > .5)*1
    
    return Y_prediction

In [8]:
# A function to define model
def model(X_train, Y_train, X_test, Y_test, iterations = 2000, learning_rate = 0.5, print_cost=False):
    NN = []
    # Define the layers here 
    add_layer(NN, X_train.shape[0], 32)
    add_layer(NN, 32, 1)
    
    NN, grads, costs = optimize(NN, X_train, Y_train, iterations, learning_rate, print_cost)

    test_predictions = predict(NN, X_test)
    train_predictions = predict(NN, X_train)

    print("train accuracy is: {} %".format(100 - np.mean(np.abs(train_predictions - Y_train)) * 100))
    print("test accuracy is: {} %".format(100 - np.mean(np.abs(test_predictions - Y_test)) * 100))
    
    results = {"costs": costs,
               "test_predictions": test_predictions, 
               "train_predictions" : train_predictions, 
               "NN" : NN,
               "learning_rate" : learning_rate,
               "iterations": iterations}
    
    return results